### DEPENDENCIES

In [9]:
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

from datasets import load_dataset
from datasets import Dataset
import pandas as pd

### IMPORT THE DATASET

In [2]:
ds = load_dataset("ai4privacy/pii-masking-400k")

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
        num_rows: 325517
    })
    validation: Dataset({
        features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
        num_rows: 81379
    })
})

In [28]:
ds["train"][0]

{'source_text': '<p>My child faozzsd379223 (DOB: May/58) will undergo treatment with Dr. faozzsd379223, office at Hill Road. Our ZIP code is 28170-6392. Consult policy M.UE.227995. Contact number: 0070.606.322.6244. Handle transactions with 6225427220412963. Queries? Email: faozzsd379223@outlook.com.</p>',
 'locale': 'US',
 'language': 'en',
 'split': 'train',
 'privacy_mask': [{'label': 'USERNAME',
   'start': 12,
   'end': 25,
   'value': 'faozzsd379223',
   'label_index': 2},
  {'label': 'DATEOFBIRTH',
   'start': 32,
   'end': 38,
   'value': 'May/58',
   'label_index': 1},
  {'label': 'USERNAME',
   'start': 72,
   'end': 85,
   'value': 'faozzsd379223',
   'label_index': 1},
  {'label': 'STREET',
   'start': 97,
   'end': 106,
   'value': 'Hill Road',
   'label_index': 1},
  {'label': 'ZIPCODE',
   'start': 124,
   'end': 134,
   'value': '28170-6392',
   'label_index': 1},
  {'label': 'TELEPHONENUM',
   'start': 180,
   'end': 197,
   'value': '0070.606.322.6244',
   'label_inde

In [27]:
ds_t = ds["train"]
ds_v = ds["validation"]

cores = multiprocessing.cpu_count()

#how to run them concurrently?
ds_t = ds_t.filter(lambda example: example["language"] == "en", num_proc = cores)
ds_v = ds_v.filter(lambda example: example["language"] == "en", num_proc = cores)

Filter (num_proc=6): 100%|██████████| 81379/81379 [00:07<00:00, 11464.15 examples/s]
